In [1]:

import pandas as pd
import numpy as np


from sqlalchemy import create_engine


ck_properties = {  # clickhouse 数据库配置
    # "driver": "com.github.housepower.jdbc.ClickHouseDriver",
    "driver": "ru.yandex.clickhouse.ClickHouseDriver",
    "socket_timeout": "300000",
    "rewriteBatchedStatements": "true",
    "batchsize": "1000000",
    "numPartitions": "8",
    "user": "default",
    "password": "5NsWWQeJ",
    "host": "10.129.165.72",
    "database": 'sdecdmp',
    "port": 8123
}


mysql_properties_prod = {  # mysql 生产环境配置
    "host": "10.129.41.147",
    "port": "4918",
    "url": "jdbc:mysql://10.129.41.147:4918/ck",
    "user": "dmp_sys_guest",
    "password": "fTe^3hy*dO_363",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

mysql_properties_dev = {  # mysql 开发环境配置
    "host": "10.179.98.13",
    "port": "3306",
    "url": "jdbc:mysql://10.179.98.13:3306/ck",
    "user": "root",
    "password": "DMP_test135#",
    "driver": "com.mysql.cj.jdbc.Driver",
    "database": "ck"
}

ck_engine = create_engine(
    f"clickhouse://{ck_properties['user']}:{ck_properties['password']}@{ck_properties['host']}:{ck_properties['port']}/{ck_properties['database']}"
)

mysql_engine_prod = create_engine(
    f"mysql+pymysql://{mysql_properties_prod['user']}:{mysql_properties_prod['password']}" +
    f"@{mysql_properties_prod['host']}:{mysql_properties_prod['port']}/{mysql_properties_prod['database']}?charset=utf8"
)

mysql_engine_dev = create_engine(
    f"mysql+pymysql://{mysql_properties_dev['user']}:{mysql_properties_dev['password']}" +
    f"@{mysql_properties_dev['host']}:{mysql_properties_dev['port']}/{mysql_properties_dev['database']}?charset=utf8"
)
                 

In [4]:
from pympler import tracker
tr = tracker.SummaryTracker()
# tr.function_without_side_effects()

In [5]:
conf = {'hos_series': 'D', 'date': '2023-05-06'}
online_ein_sql = f"""
    select B.ein1 as ein,
           B.uploadDate,
           B.cnt,
           C.hos_id,
           C.hos_series
    from (select *
          from (select if(startsWith(sma.ein, '0'), substring(sma.ein, 2, LENGTH(sma.ein)), sma.ein) as ein1
                     , any(uploadDate)                                                               as uploadDate
                     , count(1)                                                                      as cnt
                from sdecdmp.SDECData2M_all sma
                where sma.uploadDate = '{conf['date']}'
                group by ein1) A
          where A.cnt > 0
            and A.ein1 <> '') B
             left join (
        select hos_id, hos_series
        from ck_mysql.device_all
        where hos_series = '{conf['hos_series'] + '系列'}'
        ) C on B.ein1 = C.hos_id
    where C.hos_series is not null
    """
online_ein_df = pd.read_sql(
    sql=online_ein_sql,
    con=ck_engine
)
online_ein_df.sort_values(by='cnt', ascending=False)
tr.print_diff()

                                 types |   # objects |   total size
====================================== | =========== | ============
             pandas.core.series.Series |           5 |    496.36 KB
           pandas.core.frame.DataFrame |           1 |    495.80 KB
                         numpy.ndarray |          13 |     89.08 KB
                                  dict |         167 |     32.74 KB
                     collections.deque |          43 |     26.20 KB
                                   str |          94 |     16.60 KB
                                   set |           8 |     16.19 KB
                 weakref.ReferenceType |         141 |      9.91 KB
                                  code |           2 |      7.39 KB
                                  type |           1 |      6.20 KB
            builtin_function_or_method |          82 |      5.77 KB
                                  list |          38 |      2.97 KB
                                 tuple |        

In [6]:
def snat_data_loader(conf: dict, eins: list) -> str:
    """
    读取ck原始数据

    :param conf: 任务配置
    :param eins: 当日在线的车辆列表
    :return: 原始数据获取sql
    """
    # eins = snat_online_loader(date, series)
    eins_str = "'" + "', '".join(eins) + "'"

    snat_sql = f"""
    select if(startsWith(ein, '0'), substring(ein, 2, LENGTH(ein)), ein)    as ein1
        , deviceID                                                          as vin
        , formatDateTime(uploadTime1, '%Y-%m-%d %H:%M:%S')                  as uploadTime1_
        , arrayStringConcat(arrayMap(x -> toString(x), params.valueM), ',') as valueM
        , arrayStringConcat(params.codeM, ',')                              as codeM
        , longitude
        , latitude
        , uploadDate
    from sdecdmp.SDECData2M_all sma
    where
        sma.uploadDate = '{conf['date']}'
        and params.valueM is not null and params.codeM is not null
        and ein <> ''
        and ein1 in ({eins_str})
    limit 10000
    """
    return snat_sql

snat_df_sql = snat_data_loader(conf, list(online_ein_df['ein']))
snat_df_sql
tr.print_diff()

                       types |   # objects |   total size
============================ | =========== | ============
                         str |          77 |     41.26 KB
       weakref.ReferenceType |          70 |      4.92 KB
  builtin_function_or_method |          68 |      4.78 KB
                        list |          50 |      4.23 KB
                       bytes |          17 |      3.21 KB
                         set |          10 |      2.61 KB
                        dict |          13 |      2.31 KB
                        code |           3 |      2.11 KB
                       tuple |          23 |      1.34 KB
                  re.Pattern |           1 |    752     B
           collections.deque |           1 |    624     B
                        cell |          12 |    480     B
                        type |           0 |    408     B
                         int |          11 |    308     B
                    ast.Name |           6 |    288     B


In [7]:
from sqlalchemy import text

data_df = pd.read_sql(
    sql=text(snat_df_sql), con=ck_engine
)
print(data_df.head())
tr.print_diff()

          ein1                vin         uploadTime1_  \
0  D821900015C  LSCDECD821900015C  2023-05-06 17:28:17   
1  D821900015C  LSCDECD821900015C  2023-05-06 17:28:18   
2  D821900015C  LSCDECD821900015C  2023-05-06 17:28:19   
3  D821900015C  LSCDECD821900015C  2023-05-06 17:28:20   
4  D821900015C  LSCDECD821900015C  2023-05-06 17:28:21   

                                              valueM  \
0  0,52.15,1937.221,0,500,3529.411765,2,2,0,0,0,3...   
1  0,50.25,1937.221,0,500,3529.411765,2,2,0,0,0,3...   
2  0,51.1,1937.221,0,500,3529.411765,2,2,0,0,0,34...   
3  0,50.3,1937.221,0,500,3529.411765,2,2,0,0,0,34...   
4  0,50.25,1905.849,0,500,3529.411765,2,2,0,0,0,3...   

                                               codeM  longitude   latitude  \
0  SAE_J000182,SAE_J000183,DCW_U01070B,DCW_U01070...  88.058472  47.035938   
1  SAE_J000182,SAE_J000183,DCW_U01070B,DCW_U01070...  88.058484  47.035927   
2  SAE_J000182,SAE_J000183,DCW_U01070B,DCW_U01070...  88.058495  47.035915   
3 

In [10]:
gb = (sum(online_ein_df['cnt']) / 10000) * 21.38 / 1024
print(f"占用内存预估: {gb} GB")

占用内存预估: 100.98088227539061 GB
